# Twitter Sentiment Analysis

# Project Background

We will collect text data of Tweets on certain topics using Twitter API (Tweepy library) and analyse their polarity and subjectivity using Textblob library.

In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "LzsCOl12ZZ79FObx7BV9a1R3a"
consumer_secret = "gODxZ1kHJHDGgYfyP12AUN37UTlDd6yn3OsSV9Tl0iwu0K0cRY"
access_token = "296102538-kYIK8SULCmRK4vgvYD2DhKbwvSXrgTdTkWh0Nvsp"
access_token_secret = "Ib9obwgcE5tZk4S6tY5rAqI5iXOeJc9FGFg28qHoa4YpP"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

## Scraping Tweets

In [4]:
df = pd.DataFrame(columns = ['Tweets', 'User', 'User_Statuses_Count', 'User_Followers', 'Location', 'Verified', 
                             'Fav_Count', 'Rt_Count', 'Tweet_Date'])

In [5]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_Statuses_Count'] = tweet.user.statuses_count
        df.loc[i, 'User_Followers'] = tweet.user.followers_count
        df.loc[i, 'Location'] = tweet.user.location
        df.loc[i, 'Verified'] = tweet.user.verified
        df.loc[i, 'Fav_Count'] = tweet.favorite_count
        df.loc[i, 'Rt_Count'] = tweet.retweet_count
        df.loc[i, 'Tweet_Date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i+=1
        if i == 1000:
            break
        else:
            pass

In [6]:
stream(data=["Donald Trump"] , file_name='Trump')

In [7]:
df.head()

,Tweets,User,User_Statuses_Count,User_Followers,Location,Verified,Fav_Count,Rt_Count,Tweet_Date
0,RT @BBC_Culture: It has become a definitive wo...,Ernst Ekdahl,4019,12,,False,0,2,2020-10-30 06:52:20
1,"RT @JoeBiden: GDP rose last quarter, but visit...",Coach Mo,14546,122,,False,0,10807,2020-10-30 06:52:20
2,RT @PalmerReport: It's 2:30am and Donald Trump...,Rob P. #LiberateAmericavoteBidenvoteDem,405344,3532,California,False,0,20,2020-10-30 06:52:20
3,@LilTunechi @realDonaldTrump @POTUS &amp; WILL...,Michelle Renee Semak,11352,328,"Elkhart, Indiana",False,0,0,2020-10-30 06:52:19
4,"RT @MeidasTouch: We need 3,000 retweets on thi...",Sean Bernardi,1434,27,New York City,False,0,5249,2020-10-30 06:52:19


## Analysing Tweets

### To clean the tweets from extra characters

In [8]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

df['Clean_Tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

### To extract the sentiment of each tweet

In [9]:
from textblob import TextBlob
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'
    
df['Sentiment'] = df['Clean_Tweet'].apply(lambda x: analyze_sentiment(x))

In [10]:
n = 1
print("Original tweet:\n",df['Tweets'][n],'\n')
print("Clean tweet:\n",df['Clean_Tweet'][n],'\n')
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @JoeBiden: GDP rose last quarter, but visits to food banks haven’t slowed, and poverty has grown. We’re on track for the worst economic… 

Clean tweet:
 GDP rose last quarter but visits to food banks haven t slowed and poverty has grown We re on track for the worst economic 

Sentiment of the tweet:
 Negative


In [11]:
n = 20
print("Original tweet:\n",df['Tweets'][n],'\n')
print("Clean tweet:\n",df['Clean_Tweet'][n],'\n')
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @SaraGideon: Senator Collins was asked *three times* last night if she supports Donald Trump's reelection, and she refused to answer eac… 

Clean tweet:
 Senator Collins was asked three times last night if she supports Donald Trump s reelection and she refused to answer eac 

Sentiment of the tweet:
 Neutral


In [12]:
n = 12
print("Original tweet:\n",df['Tweets'][n],'\n')
print("Clean tweet:\n",df['Clean_Tweet'][n],'\n')
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @realDonaldTrump: Pollak: 11 Shining Successes of President Donald Trump‘s First Term https://t.co/VW5w4YtBQ5 via @BreitbartNews 

Clean tweet:
 Pollak 11 Shining Successes of President Donald Trump s First Term via 

Sentiment of the tweet:
 Positive


In [13]:
df[df.Sentiment == 'Positive'].shape[0]

321

In [14]:
df[df.Sentiment == 'Neutral'].shape[0]

367

In [15]:
df[df.Sentiment == 'Negative'].shape[0]

312